In [11]:
# we want to avoid using pandas to be a light weight app. But that involves reading a CSV with some work
import csv
from datetime import datetime
import os
from io import BytesIO
import pandas as pd

In [12]:
ultra_file = '/Users/naad/Downloads/Glucose_data_stint_1_25May_8Jun_UH.csv'

# Reading ultrahuman file

In [28]:
class Ultra2Normal():
    def __init__(self, filename, user_name, patient_id, date=None):
        if date:
            self.date=date
        else:
            self.date=datetime.now()
        self.filename=filename
        self.validated=False
        self.user_name=user_name
        self.patient_id=patient_id
    
    def process(self):
        self.readfile()
        file = self.write_file()
        return file
        
    def validate_file(self, header_arr):
        if len(header_arr)!=2:
            raise Exception('Columns size must be exactly 2')
            
        if header_arr[0]!='Timestamp':
            raise Exception('First column has to be a timestamp')
            
        if header_arr[1]!='Glucose Reading/Event':
            raise Exception('Second column has to be the actual readings')
        
        return True
            
        
    def readfile(self):
        rows = []
        with open(ultra_file, mode ='r')as file:
          # reading the CSV file
          csvFile = csv.reader(file)
          # displaying the contents of the CSV file
          for lines in csvFile:
                rows.append(lines)
        
        # validating header
        self.validation = self.validate_file(rows[0])
        
        self.timestamps_raw = []
        self.timestamps_out = []
        self.values = []
        self.id=[]
        self.type=[]
        # converting proper timestamps and readings
        for ii, item in enumerate(rows[1:]):
            try:
                dt_raw = datetime.strptime(item[0], '%B %d %Y, %I:%M %p')
                dt_out = dt_raw.strftime('%Y/%m/%d %H:%M')
                self.timestamps_raw.append(dt_raw)        
                self.timestamps_out.append(dt_out)        
                self.values.append(int(item[1]))
                if ii==0:
                    self.id.append(int(dt_raw.timestamp())-1680000000)
                else:
                    self.id.append(self.id[-1]+1)
                self.type.append(0)
            except Exception as e:
                print("Failed parsing this row: %d"%ii)
                print(item)
    
    def write_normal_file(self, out_path):
        data = [['ID', 'Time', 'Record Type', 'Historic Glucose (mg/dL)']]
        for ii in range(len(self.id)):
            d = [self.id[ii], self.timestamps_out[ii], self.type[ii], self.values[ii]]
            data.append(d)
        out_file = os.path.join(out_path, '%s-%s-%s.txt'%(self.patient_id, self.user_name.split(' ')[0], self.date.strftime('%d-%m-%Y')))



        with open(out_file, 'w', newline='') as file:
            file.write("%s\n"%self.user_name)
            file.write("#%s\n"%self.patient_id)
            writer = csv.writer(file, delimiter='\t')
            writer.writerows(data)

        print(f'{filename} created successfully.')
    
    def write_file(self):
        print(len(self.id), len(self.timestamps_out), len(self.type),len(self.values))
        df= pd.DataFrame({'ID':self.id,
                     'Time': self.timestamps_out[1:],
                     'Record Type':self.type,
                     'Historic Glucose (mg/dL)':self.values})
        return df.to_csv()

In [29]:
file = Ultra2Normal(ultra_file, 'temp', '123').process()

Failed parsing this row: 114
['May 25 2023,  2:45 PM', 'Lunch']
183 184 183 183


In [36]:
"%s\n%s\n%s"%('sankalp'.split(' ')[0], str(123), str('asd'))

'sankalp\n123\nasd'

# Scratch pad

In [ ]:
# !pip install streamlit
# !pip install streamlit-jupyter

In [ ]:
import streamlit as st

from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter() 

In [ ]:
if st.button('Say hello'):
    st.write('Why hello there')
else:
    st.write('Goodbye')

In [7]:
pd.DataFrame?

In [5]:
import io
import csv

# Create a BytesIO object.
byte_io = io.BytesIO()

# Create a csv.writer object for the BytesIO object.
writer = csv.writer(byte_io)

# Encode the str object to a bytes object.
data = "Name,Age\nJohn Doe,30\nJane Doe,25".encode("utf-8")

# Write the rows of the CSV file to the csv.writer object.
writer.writerow(data.splitlines())

# Get the value of the BytesIO object.
byte_io_value = byte_io.getvalue()

# Print the value of the BytesIO object.
print(byte_io_value)


TypeError: a bytes-like object is required, not 'str'